In [6]:
from lgca import get_lgca
import numpy as np
from matplotlib import pyplot as plt
%matplotlib notebook
#path = 'raw_configs/large_param_scan/'
path = '/mnt/data/simon/friedl_paper_raw_data/friedl_project/draft2/large_scan/'
params = np.load(path+'parameters.npz')
betas = params['betas']
ecms = params['ecms']
l_spheroid = params['l_spheroid']
lb = len(betas)
lecm = len(ecms)

In [7]:
vorts = np.empty((lb, lecm))
corrs = np.empty_like(vorts)
speeds = np.empty_like(corrs)
single_cells = np.empty_like(vorts)
mean_speeds = np.empty_like(vorts)

for i in range(lb):
    beta = betas[i]
    for j in range(lecm):
        ecmdens = ecms[j]

        nodes_nt = np.load(path+'nodes_nt_beta{:.1f}_ecm{:.2f}.npy'.format(beta, ecmdens))
        runs, tmax, lx, ly, K = nodes_nt.shape
        if i == 0 and j == 0:
            lgca = get_lgca(nodes=nodes_nt[0, 0])
            lgca.set_r_int(2)

        record = np.moveaxis(nodes_nt, 0, -2)
        record = np.moveaxis(record, 0, -2)
        dens = record.sum(-1)
        # single_cell = np.zeros_like(dens)


        nbs = lgca.nb_sum(dens)
        # single_cell[(nbs == 0) & (dens == 1)] = 1
        # single_cell = np.sum(single_cell, axis=(0, 1, -1))
        # single_cells[i, j] = single_cell.mean()



        flux = lgca.calc_flux(record)
        v = np.divide(flux, dens[..., None], where=dens[..., None]>0, out=np.zeros_like(flux))
        speed = np.linalg.norm(v, axis=-1)

        vort = abs(lgca.calc_vorticity(record))

        #mean_v = np.mean(speed[dens>0])
        #v = flux
        nb_v = lgca.nb_sum(v)
        nb_v_norm = np.linalg.norm(nb_v, axis=-1)

        corr = np.einsum('...i, ...i', v, nb_v)
        corr = np.divide(corr, speed, where=speed>1e-7, out=np.zeros_like(corr))
        corr = np.divide(corr, nb_v_norm, where=nb_v_norm>1e-7, out=np.zeros_like(corr))

        #corr = np.mean(corr[dens>0])


        ind = np.s_[2:-2, l_spheroid:]
        vorti = vort[ind]
        densi = dens[ind]
        nbs = nbs[ind]
        single_cell = ((nbs == 0) & (densi == 1))
        single_cell = np.sum(single_cell, axis=(0, 1, -1))
        single_cells[i, j] = single_cell.mean()
        corri = corr[ind]
        densi = dens[ind]
        speedi = speed[ind]
        speeds[i, j] = np.mean(speedi[densi>0])
        vorts[i, j] = np.mean(vorti[densi>0])
        corrs[i, j] = np.mean(corri[speedi>0])
        mean_speeds[i, j] = record[..., :lgca.velocitychannels].sum() / record.sum()

                           


Random walk interaction is used.


In [8]:
np.savez(path+'observables.npz', glob_speed=speeds, loc_speed=mean_speeds, correlations=corrs, vorticity=vorts,
         single_cells=single_cells)


In [1]:
from lgca import get_lgca
import numpy as np
from matplotlib import pyplot as plt
%matplotlib notebook
#path = 'raw_configs/large_param_scan/'
#path = '/mnt/data/simon/friedl_paper_raw_data/friedl_project/draft2/isotropic_ecm/'
path = '/home/simon/PycharmProjects/biolgca/raw_configs/isotropic_ecm/'
params = np.load(path+'parameters.npz')
betas = params['betas']
alphas = params['alphas']
l_spheroid = params['l_spheroid']
lb = len(betas)
la = len(alphas)

In [2]:
vorts = np.empty((lb, la))
corrs = np.empty_like(vorts)
speeds = np.empty_like(corrs)
single_cells = np.empty_like(vorts)
mean_speeds = np.empty_like(vorts)

for i in range(lb):
    beta = betas[i]
    for j in range(la):
        alpha = alphas[j]

        nodes_nt = np.load(path+'nodes_nt_beta{:.1f}_alpha{:.2f}.npy'.format(beta, alpha))
        runs, tmax, lx, ly, K = nodes_nt.shape
        if i == 0 and j == 0:
            lgca = get_lgca(nodes=nodes_nt[0, 0])
            lgca.set_r_int(2)

        record = np.moveaxis(nodes_nt, 0, -2)
        record = np.moveaxis(record, 0, -2)
        dens = record.sum(-1)
        # single_cell = np.zeros_like(dens)


        nbs = lgca.nb_sum(dens)
        # single_cell[(nbs == 0) & (dens == 1)] = 1
        # single_cell = np.sum(single_cell, axis=(0, 1, -1))
        # single_cells[i, j] = single_cell.mean()



        flux = lgca.calc_flux(record)
        v = np.divide(flux, dens[..., None], where=dens[..., None]>0, out=np.zeros_like(flux))
        speed = np.linalg.norm(v, axis=-1)

        vort = abs(lgca.calc_vorticity(record))

        #mean_v = np.mean(speed[dens>0])
        #v = flux
        nb_v = lgca.nb_sum(v)
        nb_v_norm = np.linalg.norm(nb_v, axis=-1)

        corr = np.einsum('...i, ...i', v, nb_v)
        corr = np.divide(corr, speed, where=speed>1e-7, out=np.zeros_like(corr))
        corr = np.divide(corr, nb_v_norm, where=nb_v_norm>1e-7, out=np.zeros_like(corr))

        #corr = np.mean(corr[dens>0])


        ind = np.s_[2:-2, l_spheroid:]
        vorti = vort[ind]
        densi = dens[ind]
        nbs = nbs[ind]
        single_cell = ((nbs == 0) & (densi == 1))
        single_cell = np.sum(single_cell, axis=(0, 1, -1))
        single_cells[i, j] = single_cell.mean()
        corri = corr[ind]
        densi = dens[ind]
        speedi = speed[ind]
        speeds[i, j] = np.mean(speedi[densi>0])
        vorts[i, j] = np.mean(vorti[densi>0])
        corrs[i, j] = np.mean(corri[speedi>0])
        mean_speeds[i, j] = record[..., :lgca.velocitychannels].sum() / record.sum()


Random walk interaction is used.


In [3]:
np.savez(path+'observables.npz', glob_speed=speeds, loc_speed=mean_speeds, correlations=corrs, vorticity=vorts,
         single_cells=single_cells)